In [1]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates.default = 'plotly_white'
import plotly
plotly.offline.init_notebook_mode(connected=True)

import pandas as pd
import numpy as np
import pickle
import os
import re

In [2]:
_nsre = re.compile('([0-9]+)')
def natural_sort_key(s):
    return [int(text) if text.isdigit() else text.lower()
            for text in re.split(_nsre, s)]

In [3]:
def conditions(name_of_file, model, mode):
    if model in name_of_file:
        if 'fit' in name_of_file:
            if 'pkl' in name_of_file:
                if mode in name_of_file:
                    return True
    return False

In [4]:
path = 'plots/fv/fits_junotao_v05d'
all_xi2 = []
all_files = os.listdir(path)
all_files.sort(key=natural_sort_key)
models = ['neural_network', 'xgboost']
modes = ['all', 'none']
for model in models:
    for mode in modes:
        xi2_array = []
        for name_of_file in all_files:
            if conditions(name_of_file, model, mode):
                with open(f'{path}/{name_of_file}', 'rb') as file:
                    readed_dict = pickle.load(file)
                    xi2 = readed_dict['fitresult']['juno']['fun']
                    success = readed_dict['fitresult']['juno']['success']
                    if success == False:
                        print(xi2, name_of_file, model, mode, )
                    xi2_array.append(xi2)
        all_xi2.append(np.array(xi2_array))

8.579824455786321 nmo-syst-subst-all-xgboost-121-fit.pkl xgboost all


In [5]:
names = ['FCDNN w. systematics', 'FCDNN w/o systematics', 'BDT w. systematics', 'BDT w/o systematics']
colors = ['darkviolet', 'darkviolet', 'darkred', 'darkred']

In [6]:
def moving_average(x, w):
    return np.convolve(x, np.ones(w), 'valid') / w

from scipy.signal import savgol_filter

In [33]:
Rs = np.linspace(16.4, 17.2, 161)
fig = go.Figure()#make_subplots(rows=2, cols=2,
#                    vertical_spacing=0.15)


# for row in range(2):
#     for col in range(2):
for i in range(4):
    x_data = np.delete(Rs, [3, 121, 144, 150])
    y_data = np.delete(all_xi2[i], [3, 121, 144, 150])
    y_data = savgol_filter(y_data, 51, 3)

    if i%2 != 0:
        dash='dash'
    else:
        dash='solid'
        
    fig.add_trace(
        go.Scatter(
            x=x_data,
            y=y_data,
            mode='lines',
            line=dict(
                dash=dash,
                color=colors[i],
            ),
            marker=dict(
                symbol='x',
                size=4,
            ),
            name=names[i],
        ),
#             row=row+1, col=col+1
    )
    
    if i%2 == 0:
        fig.add_shape(type="line",
            x0=x_data[np.argmax(y_data)], y0=8, x1=x_data[np.argmax(y_data)],
            y1=np.max(y_data),
            line=dict(
                color=colors[i],
                width=2,
                dash='dot'
            ),
        )
        
        fig.add_shape(
            type='rect',
            x0=x_data[np.argmax(y_data)]-0.022,
            x1=x_data[np.argmax(y_data)]+0.022,
            y0=np.max(y_data)+0.1,
            y1=np.max(y_data)+0.4,
            line_color='black',
            line_width=1,
            fillcolor='white'
        )

        fig.add_annotation(
            x=x_data[np.argmax(y_data)],
            y=np.max(y_data)+0.25,
            text=np.round(np.max(y_data), 3),
            showarrow=False,
            font_size=16,
            font_color='black'
        )

        fig.add_shape(
            type='rect',
            x0=17.2-0.022,
            x1=17.2+0.022,
            y0=y_data[-1]+0.1,
            y1=y_data[-1]+0.4,
            line_color='black',
            line_width=1,
            fillcolor='white'
        )

        fig.add_annotation(
            x=17.2,
            y=y_data[-1]+0.25,
            text=np.round(y_data[-1], 3),
            showarrow=False,
            font_size=16,
            font_color='black'
        )


xaxis = dict(
    showline=True,
    ticks='outside',
    mirror=True,
    tick0=16,
    dtick=0.1,
    linecolor='black',
    showgrid=True,
    gridcolor='grey',
    gridwidth=0.25,
)

yaxis = dict(
    showline=True,
    ticks='outside',
    mirror=True,
    linecolor='black',
    showgrid=True,
#     dtick=1,
    range=[8, 13],
    gridcolor='grey',
    gridwidth=0.25,
    zeroline=True,
    zerolinecolor='black',
    zerolinewidth=0.25
)

# axis_params = {}
# for i in range(1, 5):
#     axis_params[f'xaxis{i}'] = xaxis
#     axis_params[f'yaxis{i}'] = yaxis
#     axis_params[f'xaxis{i}_title'] = 'R, m'
#     axis_params[f'yaxis{i}_title'] = 'σ'

fig.update_layout(
    xaxis_title = "R, m",
    yaxis_title = "Δχ²",
    xaxis=xaxis,
    yaxis=yaxis,
    legend=dict(
#         orientation="h",
        x=0.025,
        y=0.99,
#         y=1.12,
        bordercolor="Black",
        borderwidth=1
    ),
    font=dict(
        family="Times New Roman",
        size=24,
    ),
#     **axis_params,
#     height=700
)


fig.show()
pio.write_image(fig, 'plots/fv/xi2.pdf', width=900, height=600)

In [13]:
model_index = 2
if model_index == 0:
    name='nn'
if model_index == 2:
    name='xgb'

x_data = np.delete(Rs, [3, 121, 144, 150])
y_data = np.delete(all_xi2[model_index], [3, 121, 144, 150])
y_data = savgol_filter(y_data, 51, 3)#moving_average(y_data, 5)
print(x_data[np.argmax(y_data)])

fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=x_data,
        y=y_data, 
        mode='lines',
        line=dict(
            color=colors[model_index],
        ),
        name=names[model_index],
    ),
)



fig.add_vline(
    x=x_data[np.argmax(y_data)],
    line_width=1,
    line_dash="dash",
    line_color=colors[model_index]
)

xaxis = dict(
    showline=True,
    ticks='outside',
    mirror=True,
    tick0=16,
    dtick=0.1,
    linecolor='black',
    showgrid=True,
    gridcolor='grey',
    gridwidth=0.25,
)

yaxis = dict(
    showline=True,
    ticks='outside',
    mirror=True,
    linecolor='black',
    showgrid=True,
#     dtick=0.1,
#     range=[8.1, 8.605],
    gridcolor='grey',
    gridwidth=0.25,
    zeroline=True,
    zerolinecolor='black',
    zerolinewidth=0.25
)

fig.update_layout(
    xaxis_title = "R, m",
    yaxis_title = "Δχ²",
    xaxis=xaxis,
    yaxis=yaxis,
    legend=dict(
#         orientation="h",
        x=0.025,
        y=0.99,
#         y=1.12,
        bordercolor="Black",
        borderwidth=1
    ),
    font=dict(
        family="Times New Roman",
        size=24,
    ),
#     **axis_params,
    height=600,
    width=700

)


fig.show()
pio.write_image(fig, f'plots/fv/xi2_{name}.pdf', width=600, height=600)

17.075


In [9]:
# перевзвесить. Использовать максимизацию правдоподобия